<a href="https://colab.research.google.com/github/kingy434/Sam-portfolio/blob/main/MScProject/CrossValidationPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

files.upload()

Saving Tremor_matrix_with_causal_factors_and_prototypes.mat to Tremor_matrix_with_causal_factors_and_prototypes.mat


After loading in the mat file to Colab, we then make use of scipy.io to open the mat file and create a variable for the Y features

In [3]:
from os.path import dirname, join as pjoin
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

mat_data = 'Tremor_matrix_with_causal_factors_and_prototypes.mat'
mat_contents = sio.loadmat(mat_data)
Y_features = mat_contents['Y_tremor_features_norm'];

Before loading the features into a dataframe and organising them, I first define the relevant functions I will apply to the dataframe. To begin with, for LOOCV I found it convenient to define a losses function. For the kth iteration, this function takes k as the testing set and the rest as the training set and returns the error.

In [4]:
def losses(data, algo, k, column):
  #split into 2 dataframes with k and without it
  data_filtered = data[data[column] != k]
  data_cv = data[data[column] == k]
   
  # Test data - the person left out of training
  data_test = data_cv.drop(columns=column)
  X_test = data_test.drop(columns=[outcomevar])
  y_test = data_test[outcomevar] #This is the outcome variable
      
  # Train data - all other people in dataframe
  data_train = data_filtered.drop(columns=column)
  X_train = data_train.drop(columns=[outcomevar])
      
  X_train = np.array(X_train)
  y_train = np.array(data_train[outcomevar]) #Outcome variable here

  # Train the model on training data
  algo.fit(X_train, y_train);
  predictions = algo.predict(X_test)
  return abs(predictions - y_test.values)

The following code is an adaptation for Python of Bates' R code shown here: https://github.com/stephenbates19/nestedcv/blob/master/R/core.R. It is based off the algorithm detailed in Bates, Hastie, and Tibshinari's https://arxiv.org/pdf/2104.00673.pdf. I had to make some changes for LOOCV as the code relied on making K equal partitions for each fold, and with the case of using IDs there are not 8 equal partitions for each patient. I opted to generalise this for a specific column so I could experiment with nesting on things other than the ID

In [5]:
from scipy import stats

def naive_loocv(data, algo, column, alpha = 0.1):
  """ Naive LOOCV
  Performs Naive Leave One Out Cross Validation on data provided
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - column The features who's unique values will be used as partitions
  - alpha The error rate. Must be in (0, 0.5)
  Outputs:
  - err_hat Mean of CV prediction errors
  - ci_lo Lower endpoint for CV confidence interval on prediction error 
  - ci_hi Higher endpoint for CV confidence interval on prediction error
  - sd Standard Deviation of the CV prediction errors
  """

  errors = np.array([])
  
  #get list for the values to iterate over
  vals = data[column].unique()
  for k in vals:
    #find error with k as testing subject
    error_k = losses(data, algo, k, column)
    errors = np.concatenate((errors, error_k))

    print("..." + str(int(k)) + " processing complete")

  #mean error
  err_hat = np.mean(errors)
  
  #defines the low and high confidence intervals for the error
  ci_lo = np.mean(errors) - stats.norm.ppf(1-alpha/2) * np.std(errors) / np.sqrt(len(data.index))
  ci_hi = np.mean(errors) + stats.norm.ppf(1-alpha/2) * np.std(errors) / np.sqrt(len(data.index))

  #standard deviation of errors
  sd = np.std(errors)

  #dictionary returned of relevant values
  return {'err_hat': err_hat, 'ci_lo': ci_lo, 'ci_hi': ci_hi, "sd": sd}

In [6]:
def nested_loocv_helper(data, algo, column, alpha = 0.1):
  """ Nested LOOCV helper
  Function to aid Nested LOOCV. Similar to Naive but with additional nesting stage
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - column The features who's unique values will be used as partitions
  - alpha The error rate. Must be in (0, 0.5)
  Outputs:
  - out-mat Vector of difference statistics with same length as target
  - all_ho_errors Vector of hold-out errors not used in model training
  """

  #define number of folds needed for column 
  vals = data[column].unique()
  K = len(vals)

  #fold 1 iterates through K-1 points, and fold 2 iterates through remainder of list after fold 1
  ho_errors = {}
  #entry i,j is error on fold i when folds i & j are not used for model fitting
  for f1 in range(K-1):
    for f2 in range(f1+1, K):
      ho_errors[(f1,f2)] = losses(data, algo, vals[f1], column)
      ho_errors[(f2,f1)] = losses(data[data[column] != vals[f1]], algo, vals[f2], column)

  #loops over all points in the two folds, except when the values are equal
  out_mat = np.zeros((K, 2))
  for f1 in range(K):
    e_out = losses(data, algo, vals[f1], column)

    e_bar_t = np.array([])
    for f2 in range(K):
      if f1 != f2:
        e_bar_t = np.concatenate((e_bar_t, ho_errors[(f2,f1)]))

    out_mat[f1,0] = np.mean(e_bar_t) - np.mean(e_out)
    out_mat[f1,1] = np.var(e_out) / len(data[data[column] == vals[f1]].index)

  all_ho_errors = np.array([])
  #produces final hold-out error vector for the values computed
  for f1 in range(K-1):
    for f2 in range(f1+1, K):
      all_ho_errors = np.concatenate((all_ho_errors, ho_errors[(f1,f2)], ho_errors[(f2,f1)]))

  #returns dictionary with outputs
  return {"pivots": out_mat, "errs": all_ho_errors}

In [7]:
import datetime
import math

def nested_loocv(data, algo, column, reps=50, alpha=0.1, bias_reps=None, time=False):
  """ Nested LOOCV
  Performs Nested LOOCV over several reps and computes bias
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - column The features who's unique values will be used as partitions
  - reps The number of times the algorithm should be repeated, the higher the more accurate it is
  - alpha The error rate. Must be in (0, 0.5)
  - bias_reps The number of repetitions used to calculate the bias
  - time Useful for several repetitions to get an estimate of how long it'll take to run
  Outputs:
  - sd_infl The multiplicative factor needed to correct the interval width
  - err_hat Mean of CV prediction erorrs corrected for bias
  - ci_lo Lower endpoint for CV confidence interval on prediction error 
  - ci_hi Higher endpoint for CV confidence interval on prediction error
  - raw_mean Mean of CV prediction errors without correction for bias
  - bias_est Estimation of bias from Naive LOOCV
  - sd Standard deviation of the prediction errors
  - running_sd_infl Used to evaluate how stable the estimate is and if more replicates are needed
  """

  #number of folds
  K = len(data[column].unique())

  #time estimation
  if time == True:
    t1 = datetime.datetime.now()
    temp = nested_loocv_helper(data, algo, column)
    t2 = datetime.datetime.now()
    print("Estimated time required: {}".format((t2-t1)*reps))

  #runs the algorithm several time appending the results for each repetition
  var_pivots = np.zeros((reps*K, 2))
  ho_errs = np.array([])
  for i in range(reps):
    var_pivots[K*i:K*(i+1)] = nested_loocv_helper(data, algo, column)["pivots"]
    ho_errs = np.concatenate((ho_errs, nested_loocv_helper(data, algo, column)["errs"]))
    print("...{} rep completed!".format(i))

  n_sub = math.floor((len(data[column])) * (K-1) / K)
  #estimation of inflation after each repetition
  ugp_infl = np.sqrt(max(0, np.mean(var_pivots[:,0]**2 - var_pivots[:,1]))) / (np.std(ho_errs) / np.sqrt(n_sub))
  ugp_infl = max(1, min(ugp_infl, np.sqrt(K)))

  #estimate of inflation at each time step
  infl_est2 = np.sqrt(np.maximum(0, np.array([np.mean(var_pivots[:(i+1)*K,0]**2 - var_pivots[:(i+1)*K, 1]) for i in range(reps)]))) / (np.std(ho_errs) / np.sqrt(n_sub))

  #calculates the bias using Naive LOOCV result
  cv_means = np.array([])
  bias_est = 0
  if bias_reps is None:
    bias_reps = math.ceil(reps / 5)
  if bias_reps == 0:
    bias_est = 0
  else:
    for i in range(bias_reps):
      temp = naive_loocv(data, algo, column)
      cv_means = np.append(cv_means, temp["err_hat"])
      print("...{} bias rep completed!".format(i))

    bias_est = (np.mean(ho_errs) - np.mean(cv_means)) * (1 + ((K - 2) / (K))**(1.5))

  pred_est = np.mean(ho_errs) - bias_est
  ci_lo = pred_est - stats.norm.ppf(1-alpha/2) * np.std(ho_errs) / np.sqrt(len(data[column])) * ugp_infl
  ci_hi = pred_est + stats.norm.ppf(1-alpha/2) * np.std(ho_errs) / np.sqrt(len(data[column])) * ugp_infl

  return {"sd_infl": ugp_infl, "err_hat": pred_est, "ci_lo": ci_lo, "ci_hi": ci_hi, "raw_mean": np.mean(ho_errs), "bias_est": bias_est, "sd": np.std(ho_errs), "running_sd_inf1": infl_est2}

In [8]:
def naive_tenfoldcv(data, algo, alpha=0.1):
  """ Naive 10-fold CV
  Performs Naive 10-fold Cross Validation on data provided
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - alpha The error rate. Must be in (0, 0.5)
  Outputs:
  - err_hat Mean of CV prediction errors
  - ci_lo Lower endpoint for CV confidence interval on prediction error 
  - ci_hi Higher endpoint for CV confidence interval on prediction error
  - sd Standard Deviation of the CV prediction errors
  """

  X = data.drop(columns=[outcomevar]).to_numpy()
  y = data[outcomevar].to_numpy()

  fold_id = np.array([i % 10 for i in range(len(data.index))])
  np.random.shuffle(fold_id)

  errors = np.array([])
  for k in range(10):
    algo.fit(X[fold_id != k], y[fold_id != k])
    predictions = algo.predict(X[fold_id == k])
    error_k = abs(predictions - y[fold_id == k])
    errors = np.concatenate((errors, error_k))

  err_hat = np.mean(errors)
  ci_lo = np.mean(errors) - stats.norm.ppf(1-alpha/2) * np.std(errors) / np.sqrt(len(y))
  ci_hi = np.mean(errors) + stats.norm.ppf(1-alpha/2) * np.std(errors) / np.sqrt(len(y))
  sd = np.std(errors)

  return {'err_hat': err_hat, 'ci_lo': ci_lo, 'ci_hi': ci_hi, "sd": sd}

In [9]:
def nested_tenfoldcv_helper(data, algo):
  """ Nested 10-fold CV helper
  Function to aid Nested 10-fold CV. Similar to Naive but with additional nesting stage
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - alpha The error rate. Must be in (0, 0.5)
  Outputs:
  - out-mat Vector of difference statistics with same length as target
  - all_ho_errors Vector of hold-out errors not used in model training
  """

  X = data.drop(columns=[outcomevar]).to_numpy()
  y = data[outcomevar].to_numpy()

  fold_id = np.array([i % 10 for i in range(len(y) // 10 * 10)])
  np.random.shuffle(fold_id)
  fold_id = np.concatenate((fold_id, np.repeat(10, len(y) % 10)))

  ho_errors = np.zeros((10, 10, len(y) // 10))
  for f1 in range(9):
    for f2 in range(f1+1, 10):
      test_idx = np.append(np.where(fold_id == f1), np.where(fold_id == f2))
      algo.fit(np.delete(X, test_idx, 0), np.delete(y, test_idx, 0))
      preds = algo.predict(X)
      ho_errors[f1,f2] = abs(preds[fold_id == f1] - y[fold_id == f1])
      ho_errors[f2,f1] = abs(preds[fold_id == f2] - y[fold_id == f2])

  out_mat = np.zeros((10, 2))
  for f1 in range(10):
    test_idx = np.where(fold_id == f1)
    algo.fit(np.delete(X, test_idx, 0), np.delete(y, test_idx, 0))
    preds = algo.predict(X[test_idx])
    e_out = abs(preds - y[test_idx])

    e_bar_t = np.array([])
    for f2 in range(10):
      if f1 != f2:
        e_bar_t = np.concatenate((e_bar_t, ho_errors[f2,f1]))

    out_mat[f1,0] = np.mean(e_bar_t) - np.mean(e_out)
    out_mat[f1,1] = np.var(e_out) / len(test_idx)

  all_ho_errors = np.array([])
  for f1 in range(9):
    for f2 in range(f1+1, 10):
      all_ho_errors = np.concatenate((all_ho_errors, ho_errors[f1,f2], ho_errors[f2,f1]))

  return {"pivots": out_mat, "errs": all_ho_errors}

In [10]:
def nested_tenfoldcv(data, algo, reps=50, alpha=0.1, bias_reps=None, time=False):
    """ Nested 10-fold CV
  Performs Nested 10-fold CV over several reps and computes bias
  Inputs:
  - data A dataframe of features and response variable
  - algo A classification algorithm object, with the attributes fit and predict
  - reps The number of times the algorithm should be repeated, the higher the more accurate it is
  - alpha The error rate. Must be in (0, 0.5)
  - bias_reps The number of repetitions used to calculate the bias
  - time Useful for several repetitions to get an estimate of how long it'll take to run
  Outputs:
  - sd_infl The multiplicative factor needed to correct the interval width
  - err_hat Mean of CV prediction erorrs corrected for bias
  - ci_lo Lower endpoint for CV confidence interval on prediction error 
  - ci_hi Higher endpoint for CV confidence interval on prediction error
  - raw_mean Mean of CV prediction errors without correction for bias
  - bias_est Estimation of bias from Naive LOOCV
  - sd Standard deviation of the prediction errors
  - running_sd_infl Used to evaluate how stable the estimate is and if more replicates are needed
  """

  X = data.drop(columns=[outcomevar]).to_numpy()
  y = data[outcomevar].to_numpy()

  if time == True:
    t1 = datetime.datetime.now()
    temp = nested_tenfoldcv_helper(data, algo)
    t2 = datetime.datetime.now()
    print("Estimated time required: {}".format((t2-t1)*reps))

  var_pivots = np.zeros((reps*10, 2))
  ho_errs = np.array([])
  for i in range(reps):
    var_pivots[10*i:10*(i+1)] = nested_tenfoldcv_helper(data, algo)["pivots"]
    ho_errs = np.concatenate((ho_errs, nested_tenfoldcv_helper(data, algo)["errs"]))
    print("...{} rep completed!".format(i))

  n_sub = math.floor(len(y) * 9 / 10)
  ugp_infl = np.sqrt(max(0, np.mean(var_pivots[:,0]**2 - var_pivots[:,1]))) / (np.std(ho_errs) / np.sqrt(n_sub))
  ugp_infl = max(1, min(ugp_infl, np.sqrt(10)))

  infl_est2 = np.sqrt(np.maximum(0, np.array([np.mean(var_pivots[:(i+1)*10,0]**2 - var_pivots[:(i+1)*10, 1]) for i in range(reps)]))) / (np.std(ho_errs) / np.sqrt(n_sub))

  cv_means = np.array([])
  bias_est = 0
  if bias_reps is None:
    bias_reps = math.ceil(reps / 5)
  if bias_reps == 0:
    bias_est = 0
  else:
    for i in range(bias_reps):
      temp = naive_tenfoldcv(data, algo)
      cv_means = np.append(cv_means, temp["err_hat"])
      print("...{} bias rep completed!".format(i))

    bias_est = (np.mean(ho_errs) - np.mean(cv_means)) * (1 + 0.8**1.5)

  pred_est = np.mean(ho_errs) - bias_est
  ci_lo = pred_est - stats.norm.ppf(1-alpha/2) * np.std(ho_errs) / np.sqrt(len(y)) * ugp_infl
  ci_hi = pred_est + stats.norm.ppf(1-alpha/2) * np.std(ho_errs) / np.sqrt(len(y)) * ugp_infl

  return {"sd_inf1": ugp_infl, "err_hat": pred_est, "ci_lo": ci_lo, "ci_hi": ci_hi, "raw_mean": np.mean(ho_errs), "bias_est": bias_est, "sd": np.std(ho_errs), "running_sd_inf1": infl_est2}

In [11]:
Y_features_whiten = Y_features[:,1:46]
Y_features_whiten = Y_features_whiten - np.mean(Y_features_whiten,axis=0)
from sklearn.decomposition import PCA
for i in range(45):
  pca = PCA(n_components=i+1)
  pca.fit(Y_features_whiten)
  if np.cumsum(pca.explained_variance_ratio_)[-1] > 0.95:
    opt_components = i+1
    break
Y_features_whiten = pca.fit_transform(Y_features_whiten)
Y_features_final = np.zeros((len(Y_features[:,0]), 28))
Y_features_final[:,0] = Y_features[:,0]
Y_features_final[:,1:opt_components+1] = Y_features_whiten
Y_features_final[:,opt_components+1:] = Y_features[:,46:]

data_features = pd.DataFrame(data=Y_features_final, columns=["ID", "Col2", "Col3", "Col4", "Col5", "Col6", "Col7", "Col8", "Col9", "Col10","Col11", "Col12", "Col13", "Col14", "Col15", "Col16", "Col17", "Col18", "Col19", "Col20","Col21", "Col22", "Col23", "Col24", "Medication_Intake","Prototype_ID","Non-tremor/Tremor","Activity_label"])

data_features = data_features.dropna()
data_features_id = data_features.drop(columns=["Prototype_ID", "Activity_label", "Medication_Intake"])
data_features_med = data_features.drop(columns=["Prototype_ID", "Activity_label", "ID"])
data_features_cols = data_features.drop(columns=["Prototype_ID", "Activity_label", "ID", "Medication_Intake"])

outcomevar = 'Non-tremor/Tremor'
outcomevar_id = 48
idcolumn = 'ID'
idcolumn_id = 1
medcolumn = 'Medication_Intake'

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

clf1 = RandomForestClassifier(n_estimators = 100, random_state = 0, class_weight="balanced")
clf2 = LogisticRegression(class_weight="balanced", random_state=0, solver="newton-cg")

naive_loocv_id_logreg_results = naive_loocv(data_features_id, clf2, idcolumn)
print("Results for Naive LOOCV using IDs as folds with a Logistic Regression Classifier:\n {}".format(naive_loocv_id_logreg_results))

nested_loocv_id_logreg_results = nested_loocv(data_features_id, clf2, idcolumn, reps=5, bias_reps=2)
print("\nResults for Nested LOOCV using IDs as folds with a Logistic Regression Classifier:\n {}".format(nested_loocv_id_logreg_results))

...1 processing complete
...2 processing complete
...3 processing complete
...4 processing complete
...5 processing complete
...6 processing complete
...7 processing complete
...8 processing complete
Results for Naive LOOCV using IDs as folds with a Logistic Regression Classifier:
 {'err_hat': 0.18309559009602963, 'ci_lo': 0.18077399330364438, 'ci_hi': 0.18541718688841488, 'sd': 0.38674487066206364}
...0 rep completed!
...1 rep completed!
...2 rep completed!
...3 rep completed!
...4 rep completed!
...1 processing complete
...2 processing complete
...3 processing complete
...4 processing complete
...5 processing complete
...6 processing complete
...7 processing complete
...8 processing complete
...0 bias rep completed!
...1 processing complete
...2 processing complete
...3 processing complete
...4 processing complete
...5 processing complete
...6 processing complete
...7 processing complete
...8 processing complete
...1 bias rep completed!

Results for Nested LOOCV using IDs as folds wi

In [ ]:
naive_loocv_id_rf_results = naive_loocv(data_features_id, clf1, idcolumn)
print("Results for Naive LOOCV using IDs as folds with a Random Forest Classifier:\n {}".format(naive_loocv_id_rf_results))

nested_loocv_id_rf_results = nested_loocv(data_features_id, clf1, idcolumn, reps=1, bias_reps=1)
print("\nResults for Nested LOOCV using IDs as folds with a Random Forest Classifier:\n {}".format(nested_loocv_id_rf_results))

...1 processing complete
...2 processing complete
...3 processing complete
...4 processing complete
...5 processing complete
...6 processing complete
...7 processing complete
...8 processing complete
Results for Naive LOOCV using IDs as folds with a Random Forest Classifier:
 {'err_hat': 0.10033164182682702, 'ci_lo': 0.09852811579449769, 'ci_hi': 0.10213516785915636, 'sd': 0.30044168065559795}
...0 rep completed!
...1 processing complete
...2 processing complete
...3 processing complete
...4 processing complete
...5 processing complete
...6 processing complete
...7 processing complete
...8 processing complete
...0 bias rep completed!

Results for Nested LOOCV using IDs as folds with a Random Forest Classifier:
 {'sd_infl': 2.8284271247461903, 'err_hat': 0.10010918982829801, 'ci_lo': 0.09500032149350214, 'ci_hi': 0.10521805816309389, 'raw_mean': 0.10067412908344701, 'bias_est': 0.0005649392551490062, 'sd': 0.3008967411201664, 'running_sd_inf1': array([66.61239235])}


In [14]:
naive_tenfoldcv_logreg_result = naive_tenfoldcv(data_features_cols, clf2)
print("Results for Naive 10-fold CV with a Logistic Regression Classifier:\n {}".format(naive_tenfoldcv_logreg_result))

nested_tenfoldcv_logreg_result = nested_tenfoldcv(data_features_cols, clf2, reps=5, bias_reps=2)
print("\nResults for Nested 10-fold CV with a Logistic Regression Classifier:\n {}".format(nested_tenfoldcv_logreg_result))

Results for Naive 10-fold CV with a Logistic Regression Classifier:
 {'err_hat': 0.1753839187011361, 'ci_lo': 0.17310103904777846, 'ci_hi': 0.17766679835449373, 'sd': 0.3802951482232838}
...0 rep completed!
...1 rep completed!
...2 rep completed!


/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:426: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


...3 rep completed!
...4 rep completed!
...0 bias rep completed!
...1 bias rep completed!

Results for Nested 10-fold CV with a Logistic Regression Classifier:
 {'sd_inf1': 1, 'err_hat': 0.1760213863409817, 'ci_lo': 0.1737355494586046, 'ci_hi': 0.1783072232233588, 'raw_mean': 0.17596193689693956, 'bias_est': -5.944944404213528e-05, 'sd': 0.380787780345453, 'running_sd_inf1': array([0., 0., 0., 0., 0.])}


In [ ]:
naive_tenfoldcv_rf_result = naive_tenfoldcv(data_features_cols, clf1)
print("Results for Naive 10-fold CV with a Random Forest Classifier:\n {}".format(naive_tenfoldcv_rf_result))

nested_tenfoldcv_rf_result = nested_tenfoldcv(data_features_cols, clf1, reps=1, bias_reps=1)
print("\nResults for Nested 10-fold CV with a Random Forest Classifier:\n {}".format(nested_tenfoldcv_rf_result))

Results for Naive 10-fold CV with a Random Forest Classifier:
 {'err_hat': 0.06763362235452378, 'ci_lo': 0.06612619337992215, 'ci_hi': 0.0691410513291254, 'sd': 0.25111613942900896}
Estimated time required: 0:30:39.534541
...0 rep completed!
...0 bias rep completed!

Results for Nested 10-fold CV with a Random Forest Classifier:
 {'sd_inf1': 1, 'err_hat': 0.0679084830196137, 'ci_lo': 0.06639579525407753, 'ci_hi': 0.06942117078514988, 'raw_mean': 0.06814360977919849, 'bias_est': 0.0002351267595847864, 'sd': 0.2519921788973991, 'running_sd_inf1': array([0.])}
